Generate Test Data

In [3]:
import random
import geopandas as gpd
import pandas as pd
import shapely
from shapely.geometry import Polygon, box
import h3
import time

# Generate random polygons as blobs
def generate_random_polygons(n=100, bbox=(-100, -100, 100, 100)):
    """Generate n random polygons inside a bounding box."""
    minx, miny, maxx, maxy = bbox
    polygons = []
    for _ in range(n):
        x1, y1 = random.uniform(minx, maxx), random.uniform(miny, maxy)
        x2, y2 = x1 + random.uniform(1, 5), y1 + random.uniform(1, 5)
        polygons.append(box(x1, y1, x2, y2))  # Create rectangle
    return gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")

# Create two months of blob data
prev_month_blobs = generate_random_polygons(n=100)
curr_month_blobs = generate_random_polygons(n=100)

In [ ]:
import functools

def timing_decorator(func):
    """Decorator to log function execution time."""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        elapsed_time = time.time() - start_time
        print(f"Function `{func.__name__}` took {elapsed_time:.4f} seconds.")
        return result
    return wrapper

Baseline - Polygon Matching Only